In [30]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import mutual_info_classif
from sklearn.decomposition import PCA
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import KFold
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier, RandomForestClassifier, VotingClassifier
from sklearn.metrics import confusion_matrix, precision_recall_curve, roc_curve, accuracy_score
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.mixture import GaussianMixture
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, PolynomialFeatures
from sklearn.manifold import LocallyLinearEmbedding, Isomap, TSNE
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LinearRegression, LogisticRegression, Perceptron
from sklearn.svm import LinearSVC, NuSVC, LinearSVR, NuSVR, SVC
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier, RandomForestClassifier, VotingClassifier
from sklearn.metrics import confusion_matrix, precision_recall_curve, roc_curve
from sklearn.preprocessing import StandardScaler, MinMaxScaler, PolynomialFeatures
from sklearn.manifold import LocallyLinearEmbedding, Isomap, TSNE

In [31]:
#Leemos el CSV de entrenamiento
file_path = "../traintabs/"
file_name = "traintab07.csv"
X_init = pd.read_csv(file_path+file_name, sep = ',', decimal = '.', index_col=0)

#Leemos el CSV de entrenamiento
file_path = "../traintabs/"
file_name = "train_label.csv"
Y_train_init = pd.read_csv(file_path+file_name, sep = ',', decimal = '.', index_col=0)


#Quitamos la columna de´nombre de la foto que no nos da ninguna info
X_train_init = X_init.iloc[:, 1:]
#Y_train_init.head(20)

In [32]:
class LogTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self # return get_feat_area(X)

    def transform(self, X):
        
        def get_log(features):
            numeric_columns = features.select_dtypes(include=[np.number]).columns
            log_df = pd.DataFrame()
            
            for col in numeric_columns:
                positive_values = features[col][features[col] > 0]  # Filtra solo los valores positivos de la columna
                log_df[f"log_{col}"] = np.log1p(positive_values)

            return log_df
        
        return get_log(X)

class CorrTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self # return get_feat_area(X)

    def transform(self, X):
        
        def get_cor(X_f):
            mat_R = X_f.corr()
            R_th = .97

            aux = np.abs( np.triu(mat_R.values) - np.eye(X_f.shape[1]) )
            ind_row = np.argmax(aux, axis=0)
            ind_col=0
            aboveTh_list=[]
            removed_list=[]
            for ir in ind_row:
                if (aux[ir,ind_col] >= R_th):
                    aboveTh_list.append((ir,ind_col))
                    removed_list.append(ind_col)
                ind_col=ind_col+1
            drop_list = [aux for aux in X.columns[removed_list] ]
            X_f = X_f.drop(columns=drop_list)
            X_f.head()
        return get_cor(X)

In [33]:
#PIPELINE


more_features = []

poly_degree = 2
polyfeat = PolynomialFeatures(poly_degree).set_output(transform="pandas")
more_features.append( ('addPolyFeat', polyfeat) )

#extractor = LogTransformer()
#more_features.append( ('log_transformer', extractor) )
pipe1 = Pipeline(more_features)

lastprocess = []  #--------------

#Normalizado
scaler2 = MinMaxScaler().set_output(transform='pandas')
lastprocess.append( ('scaler2', scaler2) )


##########
# Isomap #
##########

'''n_components = 8
n_neighbors = 15
metric = 'l1' #<-- 'cityblock', 'cosine', 'euclidean' , 'haversine' , 'l1' , 'l2' , 'manhattan' , 'nan_euclidean' 
max_iter = 150

isom = Isomap(n_neighbors=n_neighbors, n_components=n_components, metric=metric,
              max_iter=max_iter)

lastprocess.append( ('isom', isom) )'''
#Filtrado por corr
#cor_selector = CorrTransformer()
#lastprocess.append( ('corr_selector', cor_selector) )

pipe2 = Pipeline(lastprocess)



In [34]:
#Modelo
from sklearn.ensemble import GradientBoostingClassifier


n_estimators = 50
learning_rate= .1
choice_clf = 'linearSVC' #<-- 'linearSVC', 'SVC', 'NBC' , 'DT'
#-LinearSVC is slower other linear clfs. but it allows SAMME.R algorithm

lin_clf  = SVC(kernel='linear', C=1, probability=True)
svc_clf  = SVC(kernel='rbf', degree=3 ,C=1, probability=True)
tree_clf = DecisionTreeClassifier(max_depth=1)
nbc_clf  = GaussianNB()

if choice_clf == 'linearSVC':
    ada_clf = AdaBoostClassifier(lin_clf,
                     n_estimators=n_estimators, algorithm="SAMME.R", learning_rate=learning_rate)    
if choice_clf == 'SVC':
    ada_clf = AdaBoostClassifier(svc_clf,
                     n_estimators=n_estimators, algorithm="SAMME.R", learning_rate=learning_rate)
elif choice_clf == 'DT':
    ada_clf = AdaBoostClassifier(tree_clf,
                     n_estimators=n_estimators, algorithm="SAMME.R", learning_rate=learning_rate)
elif choice_clf == 'NBC':
    ada_clf = AdaBoostClassifier(nbc_clf,
                     n_estimators=n_estimators, algorithm="SAMME.R", learning_rate=learning_rate)

#pipeRF = Pipeline([('random', RandomForestClassifier())])


'''n_components=5
covariance_type='full' 
max_iter=15
random_state=1234

pipeRF = Pipeline([('clf_GMM', GaussianMixture(n_components=n_components, 
                                                covariance_type=covariance_type, max_iter=max_iter, random_state=random_state))])
'''
'''
hidden_layer_sizes=[32,16,8,4]
activation='tanh'        #<- ‘identity’, ‘logistic’, ‘tanh’, ‘relu’
learning_rate='adaptive' #<- 'constant’, ‘invscaling’, ‘adaptive’
learning_rate_init=0.001
max_iter=1000

pipeRF = Pipeline([('clf_NN', MLPClassifier(hidden_layer_sizes=hidden_layer_sizes,
                      activation=activation, learning_rate=learning_rate,
                      learning_rate_init=learning_rate_init, max_iter=max_iter))])
'''
'''
max_depth = 1
n_estimators = 100
learning_rate= 1

pipeRF = GradientBoostingClassifier(max_depth=max_depth,        \
                                    n_estimators=n_estimators,  \
                                    learning_rate=learning_rate)
'''
'''from sklearn.neighbors import KNeighborsClassifier
pipeRF = KNeighborsClassifier(n_neighbors=4)'''
'''n_estimators = 100
max_leaf_nodes = 16
max_depth = 1

pipeRF = RandomForestClassifier(n_estimators=n_estimators,
                                max_depth=max_depth)
                                #max_leaf_nodes=max_leaf_nodes)
'''
voting_system = 'soft' #<-- choices are 'soft' , 'hard'

lin_clf  = LogisticRegression()
svm_clf  = SVC(kernel='rbf', degree=3, C=2, probability=True)
tree_clf = DecisionTreeClassifier(max_depth=9)
nbc_clf  = GaussianNB()

pipeRF = VotingClassifier(
    estimators=[('lin', lin_clf),  ('svc', svm_clf), ('tree', tree_clf), ('nbc',nbc_clf)],
    voting=voting_system)




In [35]:
pipe = Pipeline([('pipe1',pipe1),('pipe2',pipe2),('pipeRF',pipeRF) ])
pipe

Pipeline(steps=[('pipe1',
                 Pipeline(steps=[('addPolyFeat', PolynomialFeatures())])),
                ('pipe2', Pipeline(steps=[('scaler2', MinMaxScaler())])),
                ('pipeRF', RandomForestClassifier(max_depth=1))])

In [36]:
# Calcular el índice del 80% de las filas
percentage = 0.2
split_index = int(len(X_train_init) * percentage)

# Tomar el 80% inicial de las filas como nuevo X_train
X_train = X_train_init[:split_index]
Y_train = Y_train_init[:split_index]


# Tomar el 20% restante de las filas como nuevo X_test
X_test = X_train_init[split_index:]
Y_test = Y_train_init[split_index:]


pipe.fit(X_train, Y_train)
y_hat= pipe.predict(X_test)

print('Confusion matrix:\n',pd.DataFrame(confusion_matrix(Y_test, y_hat)))
accuracy = accuracy_score(Y_test, y_hat)
print('Accuracy:', accuracy)


/home/juanmontes/.local/lib/python3.10/site-packages/sklearn/base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Confusion matrix:
       0    1
0  1121  104
1   942  101
Accuracy: 0.5388007054673721
